The Jacobian Transpose method is an inverse kinematic method that tries to find joint values to obtain an end effector position. The algorithm tries to approximate the inverse jacobian with the jacobian transpose to try and obtain a solution for the joint values.

In [105]:
import numpy as np
import sympy as sym
import math
import matplotlib.pyplot as plt

In [106]:
# SOME HARD CODED VALUES FOR ROBOT PARAMETERS
z = 0.3
l1 = 0.5
l2 = 0.5
r1 = 0.2
r2 = 0.25
r3 = 0.3
phi = 1.3
theta1_value = 1.5714
theta2_value = 0
theta3_value = 1.5714
starting_angles = sym.Matrix([theta1_value, theta2_value, theta3_value])
theta1, theta2, theta3 = sym.symbols("theta1 theta2 theta3")


In [107]:
# MAKE A DH PARAMETER TABLE
# d, alpha, r, theta
dh_parameters = sym.Matrix(
  [
   [0, 0, 0, 0],
   [z + r1, 1.57, 0, 0],
   [0, 0, r1 + l1 + r2, theta1],
   [0, 0, r2 + l2 + r3, theta2],
   [0, 0, r3, theta3]
  ] 
)
print(dh_parameters)


Matrix([[0, 0, 0, 0], [0.500000000000000, 1.57000000000000, 0, 0], [0, 0, 0.950000000000000, theta1], [0, 0, 1.05000000000000, theta2], [0, 0, 0.300000000000000, theta3]])


In [108]:

# MAKE A DH MATRIX FOR THE ROBOT

# USING THE PARAMETERS 
dh_matrix = np.diag(np.ones((4)))

print(dh_parameters.shape[0])
for i in range(dh_parameters.shape[0]):
  
  trans_z = sym.Matrix([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, dh_parameters[i, 0]],
    [0, 0, 0, 1]
  ])
  dh_matrix = dh_matrix * trans_z
  rot_z = sym.Matrix([
    [sym.cos(dh_parameters[i, 3]), -1 * sym.sin(dh_parameters[i, 3]), 0, 0],
    [sym.sin(dh_parameters[i, 3]), sym.cos(dh_parameters[i, 3]), 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0 ,1]
  ])
  dh_matrix = dh_matrix * rot_z
  trans_x = sym.Matrix([
    [1, 0, 0, dh_parameters[i, 2]],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
  ])
  dh_matrix = dh_matrix * trans_x 
  rot_x = sym.Matrix([
    [1, 0, 0, 0],
    [0, sym.cos(dh_parameters[i, 1]), -1 * sym.sin(dh_parameters[i, 1]), 0],
    [0, sym.sin(dh_parameters[i, 1]), sym.cos(dh_parameters[i, 1]), 0],
    [0, 0, 0, 1]
  ])
  dh_matrix =  dh_matrix * rot_x
  
  
print(dh_matrix) 

5
Matrix([[(-1.0*sin(theta1)*sin(theta2) + 1.0*cos(theta1)*cos(theta2))*cos(theta3) + (-1.0*sin(theta1)*cos(theta2) - 1.0*sin(theta2)*cos(theta1))*sin(theta3), -(-1.0*sin(theta1)*sin(theta2) + 1.0*cos(theta1)*cos(theta2))*sin(theta3) + (-1.0*sin(theta1)*cos(theta2) - 1.0*sin(theta2)*cos(theta1))*cos(theta3), 0, 0.3*(-1.0*sin(theta1)*sin(theta2) + 1.0*cos(theta1)*cos(theta2))*cos(theta3) + 0.3*(-1.0*sin(theta1)*cos(theta2) - 1.0*sin(theta2)*cos(theta1))*sin(theta3) - 1.05*sin(theta1)*sin(theta2) + 1.05*cos(theta1)*cos(theta2) + 0.95*cos(theta1)], [(-0.000796326710733263*sin(theta1)*sin(theta2) + 0.000796326710733263*cos(theta1)*cos(theta2))*sin(theta3) + (0.000796326710733263*sin(theta1)*cos(theta2) + 0.000796326710733263*sin(theta2)*cos(theta1))*cos(theta3), (-0.000796326710733263*sin(theta1)*sin(theta2) + 0.000796326710733263*cos(theta1)*cos(theta2))*cos(theta3) - (0.000796326710733263*sin(theta1)*cos(theta2) + 0.000796326710733263*sin(theta2)*cos(theta1))*sin(theta3), -0.999999682931

In [109]:
# GET START POSITION
start_position = dh_matrix[0:3, -1]
print(start_position)
print(start_position.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value}))

Matrix([[0.3*(-1.0*sin(theta1)*sin(theta2) + 1.0*cos(theta1)*cos(theta2))*cos(theta3) + 0.3*(-1.0*sin(theta1)*cos(theta2) - 1.0*sin(theta2)*cos(theta1))*sin(theta3) - 1.05*sin(theta1)*sin(theta2) + 1.05*cos(theta1)*cos(theta2) + 0.95*cos(theta1)], [0.3*(-0.000796326710733263*sin(theta1)*sin(theta2) + 0.000796326710733263*cos(theta1)*cos(theta2))*sin(theta3) + 0.3*(0.000796326710733263*sin(theta1)*cos(theta2) + 0.000796326710733263*sin(theta2)*cos(theta1))*cos(theta3) + 0.000836143046269926*sin(theta1)*cos(theta2) + 0.0007565103751966*sin(theta1) + 0.000836143046269926*sin(theta2)*cos(theta1)], [0.3*(-0.999999682931835*sin(theta1)*sin(theta2) + 0.999999682931835*cos(theta1)*cos(theta2))*sin(theta3) + 0.3*(0.999999682931835*sin(theta1)*cos(theta2) + 0.999999682931835*sin(theta2)*cos(theta1))*cos(theta3) + 1.04999966707843*sin(theta1)*cos(theta2) + 0.949999698785243*sin(theta1) + 1.04999966707843*sin(theta2)*cos(theta1) + 0.5]])
Matrix([[-0.301207127684100], [0.00159236469867950], [2.4996

In [110]:
# SET END POSITION

target_position = sym.Matrix([1, 1, 0.5])


In [111]:
# ALGORITHM

# CALCULATE THE JACOBIAN
angles = sym.Matrix([theta1, theta2, theta3])
# 
jacobian_matrix = start_position.jacobian(angles)

print(jacobian_matrix)

Matrix([[(0.3*sin(theta1)*sin(theta2) - 0.3*cos(theta1)*cos(theta2))*sin(theta3) + (-0.3*sin(theta1)*cos(theta2) - 0.3*sin(theta2)*cos(theta1))*cos(theta3) - 1.05*sin(theta1)*cos(theta2) - 0.95*sin(theta1) - 1.05*sin(theta2)*cos(theta1), (0.3*sin(theta1)*sin(theta2) - 0.3*cos(theta1)*cos(theta2))*sin(theta3) + (-0.3*sin(theta1)*cos(theta2) - 0.3*sin(theta2)*cos(theta1))*cos(theta3) - 1.05*sin(theta1)*cos(theta2) - 1.05*sin(theta2)*cos(theta1), -(-0.3*sin(theta1)*sin(theta2) + 0.3*cos(theta1)*cos(theta2))*sin(theta3) + (-0.3*sin(theta1)*cos(theta2) - 0.3*sin(theta2)*cos(theta1))*cos(theta3)], [(-0.000238898013219979*sin(theta1)*sin(theta2) + 0.000238898013219979*cos(theta1)*cos(theta2))*cos(theta3) + (-0.000238898013219979*sin(theta1)*cos(theta2) - 0.000238898013219979*sin(theta2)*cos(theta1))*sin(theta3) - 0.000836143046269926*sin(theta1)*sin(theta2) + 0.000836143046269926*cos(theta1)*cos(theta2) + 0.0007565103751966*cos(theta1), (-0.000238898013219979*sin(theta1)*sin(theta2) + 0.00023

In [112]:

# CALCULATE THE JACOBIAN TRANSPOSE
jacobian_transpose = jacobian_matrix.transpose()

print(jacobian_transpose)

Matrix([[(0.3*sin(theta1)*sin(theta2) - 0.3*cos(theta1)*cos(theta2))*sin(theta3) + (-0.3*sin(theta1)*cos(theta2) - 0.3*sin(theta2)*cos(theta1))*cos(theta3) - 1.05*sin(theta1)*cos(theta2) - 0.95*sin(theta1) - 1.05*sin(theta2)*cos(theta1), (-0.000238898013219979*sin(theta1)*sin(theta2) + 0.000238898013219979*cos(theta1)*cos(theta2))*cos(theta3) + (-0.000238898013219979*sin(theta1)*cos(theta2) - 0.000238898013219979*sin(theta2)*cos(theta1))*sin(theta3) - 0.000836143046269926*sin(theta1)*sin(theta2) + 0.000836143046269926*cos(theta1)*cos(theta2) + 0.0007565103751966*cos(theta1), (-0.29999990487955*sin(theta1)*sin(theta2) + 0.29999990487955*cos(theta1)*cos(theta2))*cos(theta3) + (-0.29999990487955*sin(theta1)*cos(theta2) - 0.29999990487955*sin(theta2)*cos(theta1))*sin(theta3) - 1.04999966707843*sin(theta1)*sin(theta2) + 1.04999966707843*cos(theta1)*cos(theta2) + 0.949999698785243*cos(theta1)], [(0.3*sin(theta1)*sin(theta2) - 0.3*cos(theta1)*cos(theta2))*sin(theta3) + (-0.3*sin(theta1)*cos(t

In [113]:
# GET A DIFFERENCE VECTOR

difference_position = target_position - start_position

print(difference_position)

Matrix([[-0.3*(-1.0*sin(theta1)*sin(theta2) + 1.0*cos(theta1)*cos(theta2))*cos(theta3) - 0.3*(-1.0*sin(theta1)*cos(theta2) - 1.0*sin(theta2)*cos(theta1))*sin(theta3) + 1.05*sin(theta1)*sin(theta2) - 1.05*cos(theta1)*cos(theta2) - 0.95*cos(theta1) + 1], [-0.3*(-0.000796326710733263*sin(theta1)*sin(theta2) + 0.000796326710733263*cos(theta1)*cos(theta2))*sin(theta3) - 0.3*(0.000796326710733263*sin(theta1)*cos(theta2) + 0.000796326710733263*sin(theta2)*cos(theta1))*cos(theta3) - 0.000836143046269926*sin(theta1)*cos(theta2) - 0.0007565103751966*sin(theta1) - 0.000836143046269926*sin(theta2)*cos(theta1) + 1], [-0.3*(-0.999999682931835*sin(theta1)*sin(theta2) + 0.999999682931835*cos(theta1)*cos(theta2))*sin(theta3) - 0.3*(0.999999682931835*sin(theta1)*cos(theta2) + 0.999999682931835*sin(theta2)*cos(theta1))*cos(theta3) - 1.04999966707843*sin(theta1)*cos(theta2) - 0.949999698785243*sin(theta1) - 1.04999966707843*sin(theta2)*cos(theta1)]])


In [114]:
# GET JTe and JJTe

jte = jacobian_transpose * difference_position
jjte = jacobian_matrix * jte

In [115]:
# GET ALPHA

alpha = jte.dot(jte) / jjte.dot(jjte)

print(alpha.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value}))

0.210541934291755


In [121]:
# GET DELTA THETA

delta_theta_expr = alpha * jacobian_transpose * difference_position
delta_theta = delta_theta_expr.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})

print(delta_theta)


Matrix([[-0.421058033198526], [-0.161038695575149], [0.126351098640162]])


In [122]:
print(start_position.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value}))
print(target_position)
print((jacobian_matrix * delta_theta).evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value}))

Matrix([[-0.301207127684100], [0.00159236469867950], [2.49963679772224]])
Matrix([[1], [1], [0.500000000000000]])
Matrix([[1.01104143968327], [0.000109362754582407], [0.137333984196332]])


In [117]:
# GET NEW THETA

new_theta = starting_angles + delta_theta

print (new_theta)

Matrix([[1.55197694768575], [-0.0360279538431685], [0.984804393641391]])


In [75]:
final_position_expr = start_position + (jacobian_matrix * delta_theta)
final_position = final_position_expr.evalf(subs={theta1: theta1_value, theta2: theta2_value, theta3: theta3_value})
print(final_position)

Matrix([[-0.0986433730341743], [2.16749133511659], [0.000398163355366632]])
